In [12]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [13]:
def Transform_Scale_DS(raw_data, IsTrainingData):
    df = raw_data.drop(['Sensor3', 'Sensor4', 'Sensor8','Sensor9', 'Sensor13', 'Sensor19', 'Sensor21', 'Sensor22'], axis=1)
    df = df.drop(['Sensor1', 'Sensor2'], axis=1)
    df = df.merge(df[['Engine', 'Cycles']].groupby('Engine').max(), on = 'Engine', how='left')
    df = df.rename(columns={"Cycles_x": "Cycles", "Cycles_y": "max_cycles"})
    
    # Principal Component Extraction
    pca, pcadf = PCA_Custom(df.iloc[:, 2:-2]) #Include only Sensor Data 
    pca_df = pd.DataFrame(pcadf)
    df['PC1'] = pca_df.iloc[:, 0]
    
    # Exponential Moving Average transformation on sensor data
    sens_df = df.drop(['Cycles', 'Remaining Cycles', 'max_cycles'], axis = 1)
    sens_df = sens_df.groupby('Engine').transform(lambda x: x.ewm(10).mean())
    sens_df.reset_index(inplace=True, drop = True)
    
    # Standard Scaling on features
    ss = StandardScaler()
    scaled_features = ss.fit_transform(sens_df)
    scaled_features_df = pd.DataFrame(columns = sens_df.columns, data = scaled_features)
    scaled_features_df['Cycles'] = df['Cycles']
    scaled_features_df

    # Write to CSV
    if IsTrainingData:
        scaled_features_df.to_csv('XTrain.csv', index = False)
        df['Remaining Cycles'].to_csv('YTrain.csv', index = False)
        df['Engine'].to_csv('Engine.csv', index = False)
    else:
        scaled_features_df.to_csv('XTest.csv', index = False)
        df['Remaining Cycles'].to_csv('YTest.csv', index = False)    

In [3]:
# Extract First Principal Component
def PCA_Custom(local_df):
    ss = StandardScaler()
    ss.fit(local_df)
    scaled_df = ss.transform(local_df)
    
    pca = PCA(n_components = 0.95)
    pca.fit(scaled_df)
    pca_df = pca.transform(scaled_df)
    return pca, pca_df

In [14]:
raw = pd.read_csv('E:\\NextStep\\PdM\\Predictive-Maintenance\\Datasets\\FullDataset\\train_FD001.csv')
Transform_Scale_DS(raw, 1)

In [15]:
raw = pd.read_csv('E:\\NextStep\\PdM\\Predictive-Maintenance\\Datasets\\FullDataset\\test_FD001.csv')
Transform_Scale_DS(raw, 0)